In [ ]:
%tensorflow_version 2
import os

import pandas as pd

hh_columns = ['end_t1', 'end_t2', 'req_len', 'cur_t1', 'cur_t2', 'workday',
              'rem_ch', 'rem_steps', 'const_ch', 'min_ch', 'hh_cons',  
              'hh_change', 'hh_change_1h', 'hh_change_3h', 'hh_ratio', 'target']
frames = []
for f in os.listdir('train'):
    df = pd.read_csv(f'train/{f}')
    df = df[hh_columns] # remove this line to train the model on global inputs
    frames.append(df)

train = pd.concat(map(lambda x: x[:865], frames), axis=0)
test = pd.concat(map(lambda x: x[865:], frames), axis=0)

train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

train_x = train[train.columns[:-1]]
train_y = train[train.columns[-1]]

test_x = test[test.columns[:-1]]
test_y = test[test.columns[-1]]

In [ ]:
import tensorflow as tf
import numpy as np

for i in range(10):    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['mse'])
    
    mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
    es = tf.keras.callbacks.EarlyStopping(patience=100)
    model.fit(train_x.values, train_y.values, epochs=1000, validation_data=(test_x.values, test_y.values), callbacks=[mc, es], verbose=0, batch_size=128)
    
    model = tf.keras.models.load_model('best_model.h5')
    l0 = np.append(model.layers[0].weights[1].numpy().reshape(1,-1), model.layers[0].weights[0].numpy(), axis=0).ravel()
    l1 = np.append(model.layers[1].weights[1].numpy().reshape(1,-1), model.layers[1].weights[0].numpy(), axis=0).ravel()
    #l2 = np.append(model.layers[2].weights[1].numpy().reshape(1,-1), model.layers[2].weights[0].numpy(), axis=0).ravel()

    np.save(f'weights_stable0.1_hh_50_rs_{i}.npy', np.concatenate([l0, l1]))

In [ ]:
!tar -czf weights_stable0.1_50_hh.tar.gz weights_stable0.1_hh_50_*

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

predictions = model.predict(test_x.values).reshape((-1,))

from sklearn.metrics import r2_score
R2 = r2_score(test_y, predictions, multioutput='variance_weighted')
print(R2)

plt.figure(figsize=(12,12))
plt.scatter(test_y, predictions, alpha=0.2)
plt.xlim((0,1))
plt.ylim((0,1))
plt.show()

plt.figure(figsize=(12,8))
plt.hist(test_y.values - predictions, bins=1000)
plt.xlim((-0.5, 0.5))
plt.show()